# Koneoppiminen: Tehtävä 5: Tukivektorikone  (max. 10 p)

Tukivektorikone (engl. *Support Vector Machine*, **SVM**)
* 1990-luvulla kehitetty lineaarinen luokittelumalli.

Tutki tehtävää varten myös seuraavat scikit Learn -manuaalisivut:
* [Support Vector Machines](https://scikit-learn.org/stable/modules/svm.html)
* [LinearSVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC)

In [1]:
# Kirjoita tähän tietosi!
student_name = 'Opiskelijan nimi tai Opiskelijoiden nimet'
student_id = 'JAMK-opiskelijatunnus tai JAMK-opiskelijatunnukset'

## Tehtävä 5.1
### Aihe: Syöpätutkimusaineisto (2 pistettä)
### Aineiston esikäsittely

Aineisto on lähteestä [https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Original%29](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Original%29).
* Aineistossa on yhteensä kymmenen (10) piirrettä (ominaisuutta), joiden avulla yritetään tunnistaa rintasyöpää.
* Aineistossa on luokkamuuttuja (`class`) viimeisessä sarakkeessa. Muista poistaa luokkamuuttuja `X`-alkuisista mallin syötteistä.

Toteuta tehtävä seuraavasti:

1. Lataa aineisto `DataFrame`-olioon.
2. Nimeä `DataFrame`-olion sarakkeet aineiston mukaisesti.
3. Esikäsittele aineisto toteuttaen seuraavat operaatiot
   * Arvojen muuttaminen numeerisiksi.
   * Tyhjien kenttien (`?`) korvaaminen.
5. Skaalaa koulutusaineiston sarakkeiden arvot normalisoimalla. Älä kuitenkaan koskaan skaalaa luokkamuuttujaa.

In [2]:
# TODO: Kirjoita toteutuksesi tähän soluun.
import pandas as pd
import numpy as np

df = pd.DataFrame({1:['1']})  # TODO: Alustuksen voi poistaa, kun aloittaa tehtävän tekemisen

# TODO: Esikäsittelyn toteutus

# TODO: Aineiston tulostaminen. Älä poista rivejä 
print(df.dtypes)
df.head()

1    object
dtype: object


,1
0,1


In [3]:
# Tehtävän vastaukset. Huom! Älä muokkaa tätä solua, vaan aja se, kun olet suorittanut tehtävän. 
# TODO: DataFramen tulisi olla df-muuttujassa. 
print(f'Columns: {df.columns}')
print(f'First row by position: {df.iloc[0]}')

Columns: Index([1], dtype='int64')
First row by position: 1    1
Name: 0, dtype: object


## Tehtävä 5.2
### Aihe: Koulutus- ja testausaineisto (2 pistettä)

Jaa aineisto koulutus- ja testausaineistoon käyttäen `scikit-learn`-kirjaston [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)-funktiota.
Laita aineistosta opetusaineistoon **75%**.

1. Tallenna koulutusaineisto ja sen luokkamuuttuja muuttujiin `X_train`, `y_train`,
2. Tallenna testausaineiston muuttujiin `X_test`, `y_test`.

Jos aineistossa mielestäsi tarpeettomia muuttujia, ne kannattaa poistaa tässä vaiheessa. Poistamisen voi tehdä myöhemminkin kokeilemalla ensin ajaa mallia ja vertaamalla saatujen tulosten tarkkuutta.

Käytä satunnaissiemenenä lukua `1550` ja testiaineiston kokoa **25%**.

In [4]:
# TODO: Kirjoita toteutuksesi tähän soluun.
X_test = X_train = None

In [5]:
# Tehtävän vastaukset. Huom! Älä muokkaa tätä solua, vaan aja se, kun olet suorittanut tehtävän.
X_test = X_train = pd.DataFrame()
print(len(df))
print(len(X_train))
print(len(X_test))

# df_X_train.head()

1
0
0


## Tehtävä 5.3
### Aihe: Tukivektorikone (3 pistettä)

1. Käytä `scikit-learn`-kirjaston *Linear Support Vector Classification* -funktiota [https://scikit-learn.org/stable/modules/svm.html#classification](https://scikit-learn.org/stable/modules/svm.html#classification) kouluttaaksesi koulutusaineistollasi luokittelija, joka erottelee terveet tapaukset sairaistac.

2. Tallenna ennusteet testausaineistosi datapisteille `y_pred`-muuttujaan.

3. Tee `metrics`-moduulilla [luokitteluraportti](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) testausaineistosi ennusteille. Tallenna raportti `cr`-muuttujaan. Kannattaa myös tulostaa se, jotta näet itse tulokset.

4. Tee `metrics`-moduulilla **sekaannusmatriisi** (_confusion matrix_) ja tallenna se `cm`-muuttujaan.

In [6]:
# Kirjoita toteutuksesi tähän soluun. 
cr = cm = None

In [7]:
# Tehtävän vastaukset. Huom! Älä muokkaa tätä solua, vaan aja se, kun olet suorittanut tehtävän. 
# Tulokset ovat cm- ja cr-muuttujissa. 
print(f'Classification report:\n {cr}')
print(f'Confusion matrix:\n {cm}')

Classification report:
 None
Confusion matrix:
 None


## Tehtävä 5.4
### Aihe: Vertailu toiseen menetelmään ja tulosten tulkinta (3 pistettä)

Tulkitse sanallisesti saadut testitulokset.
* Kuinka hyvin tukivektori onnistui luokittelutehtävässä? 
* Kokeile vertailun vuoksi mallin opettamista myös jollakin toisella menetelmällä.
   * Menetelmä voi olla **kNN** tai **Decision Tree** tai **Random Forest** tai mikä tahansa muut tässä tehtävässä toimiva luokittelumenetelmä.
* Piirrä myös **ROC**-käyrä opetuksen edistymisestä.
* Vertaa saatuja tuloksia ja tulkitse tulokset sanallisesti.
  * Vertaa molempien menetelmien luokittelutuloksien **tarkkuutta** sekä **sekaannusmatriisia** toisiinsa ja analysoi sanallisesti erojen syitä.

#### Toisen menetelmän toteutus
Tehdään koneoppimismallin opettaminen myös jollakin toisella menetelmällä
   * Menetelmä voi olla **kNN** tai **DecisionTree** tai **RandomForest** tai joku muu valitsemasi koneoppimismenetelmä.

In [8]:
# TODO: Kirjoita toteutuksesi tähän soluun.

# TODO: Toisen koneoppimismenetelmä toteutus
# TODO: Menetelmä voi olla kNN, DecisionTree, RandomForest tai mikä tahansa muut tässä tehtävässä toimiva menetelmä

print(f'Classification report:\n {cr}')
print(f'Confusion matrix:\n {cm}')

Classification report:
 None
Confusion matrix:
 None


#### Mallien vartailu

**Miksi tuloksia tulkitaan?** Tulosanalyysi on äärimmäisen tärkeää koneoppimisessa. Pelkkä (hyväkään) ratkaisu ei usein sellaisenaan riitä, vaan tulosten merkitys pitää ymmärtää "liiketoiminnan" kannalta.

Vertaa eri menetelmistä saatuja tuloksia.
  * vertaa menetelmien luokittelutuloksien **tarkkuutta** sekä **sekaannusmatriisia** toisiinsa.

Tulkitse tulokset myös sanallisesti.

### Vastaus: Tulosten tulkinta

* TODO: VERTAA MOLEMPIEN MENETELMIEN LUOKITTELUTULOKSIEN TARKKUUTTA SEKÄ SEKAANNUSMATRIISIA TOISIINSA.
* TODO: VERTAILUN VOI TEHDÄ SANALLISESTI SAATUJEN TULOSTEN JA KUVIEN AVULLA.
* TODO: ANALYSOI JA POHTIA SANALLISESTI SAATUJA TULOKSIA. 


#### ROC-käyrän piirtäminen

Piirrä ROC-käyrät tukivektorikoneelle ja toiselle valitulle menetelmälle.

* Pohdi mitä tuloksista voi päätellä.

In [9]:
# TODO: ROC käyrä - SVM


In [10]:
# TODO: ROC käyrä - toinen menetelmä
